# No comments / code are final

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import math

First i am importing the movement data, the index col is 0 because there are no repeating dates

In [2]:
movement_data = pd.Series(dtype=float)

movement_data = pd.read_csv("puryear_movement.csv", index_col=0)

print(movement_data)

            Distance walking / running(mi)  Flights climbed(count)  \
Date                                                                 
2020-11-21                           1.023                     5.0   
2020-11-22                           1.202                     2.0   
2020-11-23                           2.455                     8.0   
2020-11-24                           1.165                     4.0   
2020-11-25                           2.444                     3.0   
...                                    ...                     ...   
2021-11-17                           2.624                     7.0   
2021-11-18                           4.532                     7.0   
2021-11-19                           3.492                    10.0   
2021-11-20                           0.507                     4.0   
2021-11-21                           0.004                     NaN   

            Step count(count)  
Date                           
2020-11-21           2471

This next part is a bit complicated

format = `%YYYY-%MM-%DD %HH:%MM:%%SS - %YYYY-%MM-%DD %HH:%MM:%%SS,%DB_Val`

In [4]:

audio_data = pd.Series(dtype=float)

audio_data = pd.read_csv("puryear_audio.csv", index_col=0)

print(audio_data)

                                           Headphone sound levels(dBASPL)
Date                                                                     
2020-11-21 00:46:57 - 2020-11-21 00:47:48                          55.523
2020-11-21 00:47:48 - 2020-11-21 00:48:03                          51.612
2020-11-21 00:48:03 - 2020-11-21 01:01:36                          56.874
2020-11-21 11:47:18 - 2020-11-21 11:58:02                          57.440
2020-11-21 13:56:45 - 2020-11-21 13:58:10                          58.108
...                                                                   ...
2021-11-20 12:33:07 - 2021-11-20 12:33:10                          77.055
2021-11-20 12:35:07 - 2021-11-20 12:42:26                          60.440
2021-11-20 12:43:06 - 2021-11-20 12:45:21                          66.131
2021-11-21 10:11:19 - 2021-11-21 10:15:51                          60.938
2021-11-21 10:15:59 - 2021-11-21 10:17:01                          58.514

[16307 rows x 1 columns]


Format = `%YYYY-%MM-%DD %HH:%MM:%%SS - %YYYY-%MM-%DD %HH:%MM:%%SS,%DB_Val`

In [6]:
sleep_data = pd.Series(dtype=float)

sleep_data = pd.read_csv("puryear_in_bed.csv")

# i notice how whenever there is a super small one, its always after a longer one.
# my theory is that that is when i actually pick up my phone first, but then i put it down.
# maybe i hit snooze



print(sleep_data)

                                          Date  Time in bed(hr)
0    2020-11-21 01:22:35 - 2020-11-21 07:41:23            6.314
1    2020-11-22 07:24:09 - 2020-11-22 07:24:12            0.001
2    2020-11-23 01:37:06 - 2020-11-23 08:08:59            6.531
3    2020-11-23 08:11:34 - 2020-11-23 08:13:22            0.030
4    2020-11-24 03:11:00 - 2020-11-24 07:30:56            4.332
..                                         ...              ...
636  2021-11-18 04:09:32 - 2021-11-18 04:09:53            0.006
637  2021-11-18 04:20:48 - 2021-11-18 07:00:26            2.661
638  2021-11-19 00:00:00 - 2021-11-19 07:00:20            7.006
639  2021-11-20 02:08:53 - 2021-11-20 07:00:02            4.853
640  2021-11-21 00:00:00 - 2021-11-21 06:59:37            6.994

[641 rows x 2 columns]
